国土地理院の都道府県庁所在地の緯度経度のソースファイルを、より便利なcsv形式に変換したものを作成します。  
出典: 都道府県の庁舎及び東西南北端点の経緯度(世界測地系）, 国土交通省国土地理院, http://www.gsi.go.jp/KOKUJYOHO/CENTER/zenken.htm  
参考: Python3系で Webスクレイピング 入門, https://qiita.com/Senple/items/724e36fc1f66f5b14231

In [1]:
import urllib3 # urlからhtmlを取得するのに利用？
import certifi # httpsの証明書関連に必要？
from bs4 import BeautifulSoup # html解析に利用
import re # 正規表現
import pandas as pd # 表の形式に変換、csv出力に利用

# ソースからデータを取得

In [2]:
source_url = "http://www.gsi.go.jp/KOKUJYOHO/CENTER/zenken.htm" # アクセスするURL

# httpsの証明書検証を実行
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where())
r = http.request('GET', source_url)

soup = BeautifulSoup(r.data, 'html.parser') #beautifulsoupの型に変換

# 必要な部分を抽出

例えば北海道の場合、下のような形式の部分があります。
~~~
<td align="center" width="16%">
<a href="http://maps.gsi.go.jp/#16/43.06417/141.34694/&amp;base=std&amp;ls=std&amp;disp=1&amp;vs=c1j0l0u0f0">北海道庁</a>
</td>
~~~
他の都道府県も同様に、
~~~
<a href="[URL]">〇〇〇庁</a>
~~~
とURLを含めた部分があります。欲しい[緯度,経度]はこのURLの部分にあります。  
~~~
[URL]=http://maps.gsi.go.jp/#16/[緯度]/[経度]/&amp;base=std&amp;ls=std&amp;disp=1&amp;vs=c1j0l0u0f0
~~~
beautifulsoupの関数と、正規表現を使った表現でこれら[〇〇〇庁,緯度,経度]を取得します。  
全都道府県について取得し表の形式にまとめたものをcsvに出力します。

In [3]:
pref_lat_lon_list=[] # [都道府県,緯度,経度]のリスト

# <td align="center" width="16%">の部分を見る
for td in soup.find_all("td",align="center",width="16%"):
    pref=re.findall(r">.{3,4}庁<",str(td)) # 都道府県を取得
    
    # もし都道府県を取得できれば（都道府県が含まれていない箇所もあるため）
    if pref:
        pref="".join(pref[0][1:-1]) # 都道府県名
        map_url=re.findall(r"http[s]*://.*/.*/.*/.*/&amp",str(td))[0] # http://~/の部分を抽出
        map_url=map_url.split("//")[1] # http://の後ろを抽出
        lat,lon=list(map(float, map_url.split("/")[2:4])) # 緯度、経度を抽出
        
        pref_lat_lon_list.append([pref,lat,lon]) # リストに追加

以上でリストの形で[都道府県,緯度,経度]を得られました。  
csv出力をする際にはpandas.DataFrameの形式に変換すると便利です。

In [4]:
P=pd.DataFrame(pref_lat_lon_list,columns=["都道府県","緯度","経度"]) # データフレーム型に変換
print("P.shape = ",P.shape)
P.head(10)

P.shape =  (47, 3)


,都道府県,緯度,経度
0,北海道庁,43.06417,141.34694
1,青森県庁,40.82444,140.74000
2,岩手県庁,39.70361,141.15250
3,宮城県庁,38.26889,140.87194
4,秋田県庁,39.71861,140.10250
5,山形県庁,38.24056,140.36333
6,福島県庁,37.75000,140.46778
7,茨城県庁,36.34139,140.44667
8,栃木県庁,36.56583,139.88361
9,群馬県庁,36.39111,139.06083


# csv出力

それではこのデータフレームをcsv出力します。

In [5]:
P.to_csv("./都道府県庁所在地_緯度経度.csv",index=False) # .csv出力

# 出力確認

念のため、正しく保存できたかどうかを確認します。

In [6]:
pd.read_csv("./都道府県庁所在地_緯度経度.csv") # csvを読み込み

,都道府県,緯度,経度
0,北海道庁,43.06417,141.346940
1,青森県庁,40.82444,140.740000
2,岩手県庁,39.70361,141.152500
3,宮城県庁,38.26889,140.871940
4,秋田県庁,39.71861,140.102500
5,山形県庁,38.24056,140.363330
6,福島県庁,37.75000,140.467780
7,茨城県庁,36.34139,140.446670
8,栃木県庁,36.56583,139.883610
9,群馬県庁,36.39111,139.060830


確認できました。